#Installing pykeen-AND

In [ ]:
cd "drive/My Drive/thesis_project/31-05-2021/pykeen-AND"

/content/drive/My Drive/thesis_project/31-05-2021/pykeen


Execute the following command line inside the folder in which your modified pykeen library is.

In [ ]:
!pip install -e .

Obtaining file:///content/drive/My%20Drive/thesis_project/31-05-2021/pykeen
     |████████████████████████████████| 27.4MB 194kB/s 
     |████████████████████████████████| 307kB 24.0MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 174kB 52.3MB/s 
     |████████████████████████████████| 81kB 11.9MB/s 
     |████████████████████████████████| 81kB 11.8MB/s 
     |████████████████████████████████| 112kB 60.6MB/s 
     |████████████████████████████████| 143kB 52.2MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
  Created wheel for click-default-group: filename=click_default_group-1.2.2-cp37-none-any.whl size=3386 sha256=f420adbacc7de1ecb1f4224929dabf7a9f77604cc8e81c0d1e83a08af8b69506
  Stored in directory: /root/.cache/pip/wheels/ca/ed/79/2200269c285b0d21ab4591c6388a14691a4fc77d76bec40fe2
  Created wheel for stringcase: filename=stringcase-1.2.0-cp37-none-any.whl size=3588 sha256=e80af82e525afb3f000c80287f2d818175aa44f76878dac1f3

In [ ]:
cd ../

/content/drive/My Drive/thesis_project/31-05-2021


Install the following dependency only if you need to use **DistMultText** class.

In [ ]:
!pip install sentence-transformers

     |████████████████████████████████| 81kB 2.3MB/s 
     |████████████████████████████████| 2.3MB 4.2MB/s 
     |████████████████████████████████| 1.2MB 35.8MB/s 
     |████████████████████████████████| 3.3MB 37.4MB/s 
     |████████████████████████████████| 901kB 38.3MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.2.0-cp37-none-any.whl size=123339 sha256=d693f1e2eeb2071c885a2f8a3c4b5d2cf591163c5a94010dd40893f2e6859d72
  Stored in directory: /root/.cache/pip/wheels/0f/06/f7/faaa96fdda87462b4fd5c47b343340e9d5531ef70d0eef8242
Successfully built sentence-transformers


# Pykeen import bug

Pykeen suffers from a little import bug on colab so once you have installed your dependecies you should restart the runtime.

In [ ]:
import pykeen

In [ ]:
cd "../content/drive/My Drive/thesis_project/31-05-2021/"

/content/drive/My Drive/thesis_project/31-05-2021


# Import dataset

This part is a little tricky.
We will use the built-in class **TriplesFactory** from pykeen to convert our TSV dataset into a numpy array of triples whose entities and relations are mapped to a set of IDs. We will then use the TriplesFactory built-in method *.split()* to split the dataset into training, testing and validation, making sure that there's no unseen triple in the validation and test set.
However, and here it becomes tricky, since our **DistMultText** model accepts as training triples only the class **TriplesTextualLiteralsFactory**, which provides entity descriptions in the form of embeddings, we need to convert the training triples into the forementioned class.
Since instatiating a new TriplesTextualLiteralsFactory remaps every entity and relation to a new set of IDs, we need to reinstatiate both the test and the validation dataset to new TriplesFactory classes, which we will then take as argument the entity and relation to IDs mapping of the training dataset. 

In [ ]:
from pykeen.triples import TriplesFactory
tf = TriplesFactory.from_path(path="OC-197K/kg.txt")

In [ ]:
training, testing, validation = tf.split([.8, .1, .1])

using automatically assigned random_state=2890984789


In [ ]:
from pykeen.triples import TriplesTextualLiteralsFactory
training = TriplesTextualLiteralsFactory(triples=training.triples, path_to_textual_triples="OC-197K/text_literals.txt") 

Reconstructing all label-based triples. This is expensive and rarely needed.
87511it [15:59, 91.18it/s]


In [ ]:
testing = TriplesFactory.from_labeled_triples(triples=testing.triples, entity_to_id=training.entity_to_id, relation_to_id=training.relation_to_id)
validation = TriplesFactory.from_labeled_triples(triples=validation.triples, entity_to_id=training.entity_to_id, relation_to_id=training.relation_to_id)

Reconstructing all label-based triples. This is expensive and rarely needed.
Reconstructing all label-based triples. This is expensive and rarely needed.


# Run Hyper-parameter optimization

In [38]:
from pykeen.hpo import hpo_pipeline

hpo_result = hpo_pipeline(
    n_trials=100,
    training=training,
    testing=testing,
    validation=validation,
    model='DistMultText',
    model_kwargs_ranges=dict(
        embedding_dim=dict(type=int, low=100, high=500, q=100),
    ),
    optimizer_kwargs_ranges=dict(
        lr=dict(type='float', low=0.001, high=0.1, scale="log")
    ),
    stopper = "early",
    stopper_kwargs = {
        
        "frequency": 5,
        "patience": 20,
        "relative_delta": 0.002,
        "metric": "hits@10"
    },
    timeout=21600
)
hpo_result.save_to_directory("./hpo_results")

[I 2021-06-07 14:29:59,677] A new study created in memory with name: no-name-89c43843-598f-47ce-8f8b-bee849e4b980
/usr/local/lib/python3.7/dist-packages/optuna/distributions.py:563: UserWarning:

The distribution is specified by [32, 4000] and step=100, but the range is not divisible by `step`. It will be replaced by [32, 3932].

No random seed is specified. Setting to 1751960199.


INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...
INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=128.
INFO:pykeen.evaluation.evaluator:Evaluation took 17.78s seconds
INFO:pykeen.training.training_loop:=> Saved checkpoint after having finished epoch 5.


INFO:pykeen.evaluation.evaluator:Evaluation took 17.48s seconds
INFO:pykeen.training.training_loop:=> Saved checkpoint after having finished epoch 10.


INFO:pykeen.evaluation.evaluator:Evaluation took 17.49s seconds
INFO:pykeen.training.training_loop:=> Saved checkpoint after having finished epoch 15.


INFO:pykeen.evaluation.evaluator:Evaluation took 17.48s seconds


...

INFO:pykeen.evaluation.evaluator:Evaluation took 11.26s seconds


INFO:pykeen.evaluation.evaluator:Evaluation took 11.28s seconds


INFO:pykeen.evaluation.evaluator:Evaluation took 11.25s seconds


INFO:pykeen.evaluation.evaluator:Evaluation took 11.27s seconds
INFO:pykeen.stoppers.early_stopping:Stopping early after 30 evaluations at epoch 150. The best result hits@10=0.04150737302020754 occurred at epoch 50.
INFO:pykeen.training.training_loop:=> loading checkpoint '/tmp/tmptujnpa28'
INFO:pykeen.training.training_loop:=> loaded checkpoint '/tmp/tmptujnpa28' stopped after having finished epoch 50


INFO:pykeen.evaluation.evaluator:Evaluation took 11.25s seconds
[I 2021-06-07 20:45:07,675] Trial 19 finished with value: 0.6681292095492906 and parameters: {'model.embedding_dim': 100, 'model.input_dropout': 0.30000000000000004, 'loss.margin': 2, 'loss.margin_activation': 'softplus', 'optimizer.lr': 0.09862774090894078, 'negative_sampler.num_negs_per_pos': 45, 'training.num_epochs': 600, 'training.batch_size': 3532}. Best is trial 10 with value: 0.9129438248213196.
